In [2]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

# Pasta atual do projeto
current_folder = os.getcwd()

# Diretório dos dados
DATA_DIR = os.path.join(current_folder, "data")

# Cria o diretório se não existir
os.makedirs(DATA_DIR, exist_ok=True)

# Verifica se o diretório de dados está vazio
if not os.listdir(DATA_DIR):
    print("Diretório está vazio. Baixando dataset...")

    # Inicializa e autentica na API do Kaggle
    api = KaggleApi()
    api.authenticate()

    # Faz o download e extrai arquivos
    api.dataset_download_files('uciml/adult-census-income',  # nome do dataset desejado
        path=DATA_DIR,
        unzip=True
    )

    print("Download concluído com sucesso!")
else:
    print("Diretório já contém arquivos. Nenhuma ação necessária.")

Diretório já contém arquivos. Nenhuma ação necessária.


In [ ]:
# importar os pacotes necessários
from ast import ImportFrom
import pandas as pd

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [8, 6]
import pdb
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
from utils.eda import *
from utils.feature_engineering import *
#teste
#teste
# filtrar mensagens de warning
import warnings
warnings.filterwarnings('ignore')

In [4]:
##Carregando os datasets de treino na fonte de dados
treino_path='C:/Users/Sergio/OneDrive/FIAP/Tech_Challenge_Fase_3/data/adult.csv'
df_treino=pd.read_csv(treino_path, names=['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'], header=None, index_col=False)
pd.set_option('display.max_columns', 500)
display(df_treino)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
1,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
2,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
3,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
4,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32557,22,Private,310152,Some-college,10,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,United-States,<=50K
32558,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32559,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32560,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K


In [5]:
#Imprimindo as 5 primeiras linhas
df_treino.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
1,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
2,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
3,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
4,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K


In [6]:
#Imprimindo as 5 últimas linhas
df_treino.tail()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
32557,22,Private,310152,Some-college,10,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,United-States,<=50K
32558,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32559,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32560,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32561,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [7]:
#Imprimindo a quantidade de dados e variáveis
print("Quantidade de dados: ", df_treino.shape[0])
print("Quantidade de variáveis: ", df_treino.shape[1])

Quantidade de dados:  32562
Quantidade de variáveis:  15


In [8]:
import pandas as pd

# Supondo que o dataframe original se chame 'df'
# Converta as variáveis numéricas para int64:
numeric_vars = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
df_treino[numeric_vars] = df_treino[numeric_vars].apply(pd.to_numeric, errors='coerce').astype('Int64')

# Converta as variáveis categóricas para object (string):
categorical_vars = ['workclass', 'education', 'marital-status', 
                    'occupation', 'relationship', 'race', 
                    'sex', 'native-country', 'income']

df_treino[categorical_vars] = df_treino[categorical_vars].astype(str)

In [9]:
#Armazenando as features
features = df_treino.columns.to_list()

#Armazenando as informações das variáveis
df_var = Map_Var_DF(features = features, df = df_treino)

#Imprimindo o dataframe
df_var

,feature,Tipo,Categórico,Binário,Qtd var unico,Min,Max,% Qtd de Nulos
0,age,Int64,0,0,73,17,90,0.0
1,workclass,object,1,0,10,N/A,N/A,0.0
2,fnlwgt,Int64,0,0,21648,12285,1484705,0.0
3,education,object,1,0,17,N/A,N/A,0.0
4,education-num,Int64,0,0,16,1,16,0.0
5,marital-status,object,1,0,8,N/A,N/A,0.0
6,occupation,object,1,0,16,N/A,N/A,0.0
7,relationship,object,1,0,7,N/A,N/A,0.0
8,race,object,1,0,6,N/A,N/A,0.0
9,sex,object,1,0,3,N/A,N/A,0.0


In [10]:
descricao_dataset(df_treino)

Quantidade de dados:  (32562, 15)

Quantidade de variáveis: Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')

Dataframe primeira ate a ultima linha:



,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,<NA>,workclass,<NA>,education,<NA>,marital.status,occupation,relationship,race,sex,<NA>,<NA>,<NA>,native.country,income
1,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
2,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
3,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
4,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32557,22,Private,310152,Some-college,10,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,United-States,<=50K
32558,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32559,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32560,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K



Descrição das Colunas:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32562 entries, 0 to 32561
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  Int64 
 1   workclass       32562 non-null  object
 2   fnlwgt          32561 non-null  Int64 
 3   education       32562 non-null  object
 4   education-num   32561 non-null  Int64 
 5   marital-status  32562 non-null  object
 6   occupation      32562 non-null  object
 7   relationship    32562 non-null  object
 8   race            32562 non-null  object
 9   sex             32562 non-null  object
 10  capital-gain    32561 non-null  Int64 
 11  capital-loss    32561 non-null  Int64 
 12  hours-per-week  32561 non-null  Int64 
 13  native-country  32562 non-null  object
 14  income          32562 non-null  object
dtypes: Int64(6), object(9)
memory usage: 3.9+ MB


None

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32561.0,32561.0,32561.0,32561.0,32561.0,32561.0
mean,38.581647,189778.366512,10.080679,1077.648844,87.30383,40.437456
std,13.640433,105549.977697,2.57272,7385.292085,402.960219,12.347429
min,17.0,12285.0,1.0,0.0,0.0,1.0
25%,28.0,117827.0,9.0,0.0,0.0,40.0
50%,37.0,178356.0,10.0,0.0,0.0,40.0
75%,48.0,237051.0,12.0,0.0,0.0,45.0
max,90.0,1484705.0,16.0,99999.0,4356.0,99.0



Valores Nulos:



age               1
workclass         0
fnlwgt            1
education         0
education-num     1
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      1
capital-loss      1
hours-per-week    1
native-country    0
income            0
dtype: int64

In [ ]:
variavel='income'
analise_variavel_categorica(df_treino, variavel)
##plot de distribuição
ax=df_treino[variavel].value_counts().plot.bar()
ax.set_title('Distribuição dos dados da variavel '+variavel)
ax.set_ylabel('Numero de pessoas')
ax.set_xlabel('income  anual')
ax.set_xticklabels(labels=[' <=50K', ' >50K'])

NameError: name 'analise_variavel_categorica' is not defined

In [ ]:
variavel='age'
analise_variavel_categorica(df_treino, variavel)


ax=df_treino[variavel].hist(bins=30)
ax.set_title('Distribuição dos dados da variavel '+variavel)
ax.set_ylabel('Numero de pessoas')
ax.set_xlabel('Age')
plt.show()

sns.boxplot(y='age' , x='class' , data=df_treino)
plt.show()

In [ ]:

variavel='fnlwgt'
analise_variavel_categorica(df_treino, variavel)


ax=df_treino[variavel].hist(bins=100)
ax.set_title('Distribuição dos dados da variavel '+variavel)
ax.set_ylabel('Numero de pessoas')
ax.set_xlabel('fnlwgt')
plt.show()

sns.boxplot(y='fnlwgt' , x='class' , data=df_treino)
plt.show()

In [ ]:
variavel='education'
analise_variavel_continua(df_treino, variavel)

##Plot da distribuição
ax=df_treino[variavel].value_counts(normalize=True).plot.bar()
ax.set_title('Distribuição dos dados da variavel '+variavel)
ax.set_ylabel('Porcentagem de pessoas')
ax.set_xlabel('Education')
ax.set_xticklabels(labels=[' HS-grad', ' Some-college', ' Bachelors', ' Masters', ' Assoc-voc', ' 11th', ' Assoc-acdm', ' 10th', ' 7th-8th', ' Prof-school', ' 9th', ' 12th', ' Doctorate', ' 5th-6th', ' 1st-4th', ' Preschool'])
plt.show()

print('Porcentagem de pessoas com  HS-grad that makes >$50K/yr: ', len(df_treino[(df_treino['education']==' HS-grad') & (df_treino['class']==' >50K')])/len(df_treino)*100)
print('Porcentagem de pessoas com college degree que fazem >$50K/yr: ', len(df_treino[(df_treino['education']==' Some-college') & (df_treino['class']==' >50K')])/len(df_treino)*100)
print('Porcentagem de pessoas com bachelors que fazem  >$50K/yr: ', len(df_treino[(df_treino['education']==' Bachelors') & (df_treino['class']==' >50K')])/len(df_treino)*100)
print('Porcentagem de pessoas com doctorate degree que fazem  >$50K/yr: ', len(df_treino[(df_treino['education']==' Doctorate') & (df_treino['class']==' >50K')])/len(df_treino)*100)

display(pd.crosstab([df_treino['education']], df_treino['class'], normalize='index'))

pd.crosstab([df_treino['education']], df_treino['class'], normalize='index').plot.bar()
plt.ylabel('Porcentagem de pessoas  (Normalizado pela class)')

In [ ]:
variavel='relationship'
analise_variavel_continua(df_treino, variavel)

##Plot da distribuição
ax=df_treino[variavel].value_counts(normalize=True).plot.bar()
ax.set_title('Distribuição dos dados da variavel '+variavel)
ax.set_ylabel('Porcentagem de pessoas')
ax.set_xlabel('relationship')
ax.set_xticklabels(labels=[' Husband', ' Not-in-family', ' Own-child', ' Unmarried', ' Wife', ' Other-relative'])
plt.show()

display(pd.crosstab([df_treino['relationship']], df_treino['class'], normalize='index'))

pd.crosstab([df_treino['relationship']], df_treino['class'], normalize='index').plot.bar()
plt.ylabel('Porcentagem de pessoas  (Normalizado pela class)')

In [ ]:
variavel='race'
analise_variavel_categorica(df_treino, variavel)

##plot distribution
ax=df_treino[variavel].value_counts(normalize=True).plot.bar()
ax.set_title('Distribuição dos dados da variavel '+variavel)
ax.set_ylabel('Percentagem de people')
ax.set_xlabel('Race')
ax.set_xticklabels(labels=[' White', ' Black', ' Asian-Pac-Islander', ' Amer-Indian-Eskimo', ' Other'])
plt.show()

display(pd.crosstab([df_treino['race']], df_treino['class'], normalize='index'))

pd.crosstab([df_treino['race']], df_treino['class'], normalize='index').plot.bar()
plt.ylabel('Porcentagem de pessoas  (normalizado pela class)')

In [ ]:
variavel='sex'
analise_variavel_categorica(df_treino, variavel)

##Plot de distribuição
ax=df_treino[variavel].value_counts(normalize=True).plot.bar()
ax.set_title('Distribuição dos dados da variavel '+variavel)
ax.set_ylabel('Percentagem de people')
ax.set_xlabel('sex')
ax.set_xticklabels(labels=[' Male', ' Female'])
plt.show()

display(pd.crosstab([df_treino['sex']], df_treino['class'], normalize='index'))

pd.crosstab([df_treino['sex']], df_treino['class'], normalize='index').plot.bar()
plt.ylabel('Porcentagem de pessoas  (normalizado pela class)')

In [ ]:

variavel='capital-gain'
analise_variavel_continua(df_treino, variavel)

ax=df_treino[variavel].hist(bins=50)
ax.set_title('Distribuição de dados da variavel '+variavel)
ax.set_ylabel('Numero de pessoas')
ax.set_xlabel('Capital gain')
plt.show()

sns.boxplot(y='capital-gain', x='class', data=df_treino)
plt.show()

In [ ]:
variavel='capital-loss'
analise_variavel_continua(df_treino, variavel)

ax=df_treino[variavel].hist(bins=50)
ax.set_title('Distribuição de dados da variavel '+variavel)
ax.set_ylabel('Numero de pessoas')
ax.set_xlabel('Capital loss')
plt.show()

sns.boxplot(y='capital-loss', x='class', data=df_treino)
plt.show()

In [ ]:
variavel='hours-per-week'
analise_variavel_continua(df_treino, variavel)

ax=df_treino[variavel].hist(bins=50)
ax.set_title('Distribuição de dados da variavel '+variavel)
ax.set_ylabel('Numero de pessoas')
ax.set_xlabel('hours per week')
plt.show()

sns.boxplot(y='hours-per-week', x='class', data=df_treino)
plt.show()

print('Porcentagem de pessoas que trabalham  Menos que 40 horas por semana e fazem  >$50K/yr: ', len(df_treino[(df_treino['hours-per-week']<40) & (df_treino['class']==' >50K')])/len(df_treino)*100)
print('Porcentagem de pessoas que trabalham  Menos que 40 horas por semana e fazem  <=$50K/yr: ', len(df_treino[(df_treino['hours-per-week']<40) & (df_treino['class']==' <=50K')])/len(df_treino)*100)
print('Porcentagem de pessoas que trabalham  Entre 40 and 45 horas por semana e fazem  >$50K/yr: ', len(df_treino[((df_treino['hours-per-week'] >= 40) & (df_treino['hours-per-week'] <= 45)) & (df_treino['class']==' >50K')])/len(df_treino)*100)
print('Porcentagem de pessoas que trabalham  Entre 40 and 45 horas por semana e fazem  <=$50K/yr: ', len(df_treino[((df_treino['hours-per-week'] >= 40) & (df_treino['hours-per-week'] <= 45)) & (df_treino['class']==' <=50K')])/len(df_treino)*100)
print('Porcentagem de pessoas que trabalham  Mais que 45 horas por semana e fazem  >$50K/yr: ', len(df_treino[(df_treino['hours-per-week'] > 45) & (df_treino['class']==' >50K')])/len(df_treino)*100)
print('Porcentagem de pessoas que trabalham  Mais que 45 horas por semana e fazem  <=$50K/yr: ', len(df_treino[(df_treino['hours-per-week'] > 45) & (df_treino['class']==' <=50K')])/len(df_treino)*100)

In [ ]:
variavel='native-country'
analise_variavel_categorica(df_treino, variavel)

##plot distribution
ax=df_treino[variavel].value_counts(normalize=True).plot.bar()
ax.set_title('Distribuição de dados da variavel '+variavel)
ax.set_ylabel('Porcentagem de pessoas')
ax.set_xlabel('Native country')
ax.set_xticklabels(labels=df_treino['native-country'].unique().tolist())
plt.show()

pd.crosstab([df_treino['native-country']], df_treino['class'], normalize='index') #normaliza em cada linha

In [ ]:
display(df_treino.groupby(['sex', 'race', 'class']).mean())

In [ ]:

display(df_treino.groupby(['occupation', 'class']).mean())

In [ ]:
##categorical features
df_treino=pd.get_dummies(df_treino, columns=['class', 'sex'], drop_first=True) #Transformação binaria

df_treino['workclass_simplificada']=df_treino['workclass'].apply(lambda variavel:' WO_pay' if variavel==' Without-pay' or variavel==' Never-worked' else variavel )

df_treino['occupation_simplificada']=df_treino['occupation'].apply( lambda variavel:' Other-service' if variavel==' Armed-Forces' or variavel=='?' or variavel=='Other-service' else variavel)

def status_married(variavel):
  #Combinação das classes similares
  if variavel==' Married-civ-spouse' or variavel==' Married-AF-spouse':
    return ' Married'
  elif variavel==' Never-married':
    return variavel
  else:
    return ' No-spouse'

df_treino['marital-status_simplificada']=df_treino['marital-status'].map(status_married)



def grupos_race(variavel):
  #Agrupando algumas classes
  if variavel==' White':
    return variavel
  else:
    return ' Other'

df_treino['race_simplificada']=df_treino['race'].map(grupos_race)

def grupos_relacionamento(variavel):
  #Agrupando Classes similares
  if variavel==' Husband' or variavel==' Wife':
    return ' Spouse'
  else:
    return variavel
df_treino['relationship_simplificada']=df_treino['relationship'].map(grupos_relacionamento)


def grupos_paises(val):
  #Agrupando paises em continentes
  if val==' United-States':
    return val
  elif val in [' Mexico', ' Cuba', ' Jamaica', ' Puerto-Rico', ' Honduras', ' Columbia', ' Ecuador', ' Guatemala', ' Peru', ' Outlying-US(Guam-USVI-etc)', ' Nicaragua', ' Haiti',
               ' Dominican-Republic', ' El-Salvador', ' Trinadad&Tobago', ' South']:
    return ' America'
  elif val in [' England', ' Canada', ' Germany', ' Italy', ' Poland',  ' Portugal', ' France', ' Greece',  ' Ireland', ' Hungary', ' Yugoslavia', ' Holand-Netherlands',
               ' Scotland']:
    return ' Europe'
  elif val in [' India', ' Philippines', ' Iran', ' Cambodia', ' Thailand', ' Laos', ' Taiwan', ' China', ' Japan', ' Vietnam', ' Hong']:
    return ' Asia'
  else:
    return ' ?'

df_treino['native-country_simplificada']=df_treino['native-country'].map(grupos_paises)


df_treino=pd.get_dummies(df_treino, columns=['workclass_simplificada', 'occupation_simplificada', 'marital-status_simplificada', 'relationship_simplificada', 'race_simplificada', 'native-country_simplificada'], drop_first=True)

In [ ]:
variavel=['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

df_treino=padronizacao_variavel(df_treino,variavel)

In [ ]:
df_treino.head(5)

In [ ]:
correlacao=df_treino.corr()
mask = np.triu(np.ones_like(correlacao, dtype=bool))
# Configurando a figura do matplotlib
f, ax = plt.subplots(figsize=(25, 20))
# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)
plot = sns.heatmap(correlacao, mask=mask, cmap=cmap, center=0,
            square=False, linewidths=.5, cbar_kws={"shrink": .9}, annot=True, fmt='.2f'  )

In [ ]:
print(df_treino.columns)

variaveis=['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week',
       'sex_ Male',
       'workclass_simplificada_ Federal-gov',
       'workclass_simplificada_ Local-gov',
       'workclass_simplificada_ Private',
       'workclass_simplificada_ Self-emp-inc',
       'workclass_simplificada_ Self-emp-not-inc',
       'workclass_simplificada_ State-gov',
       'workclass_simplificada_ WO_pay',
       'occupation_simplificada_ Craft-repair',
       'occupation_simplificada_ Exec-managerial',
       'occupation_simplificada_ Farming-fishing',
       'occupation_simplificada_ Handlers-cleaners',
       'occupation_simplificada_ Machine-op-inspct',
       'occupation_simplificada_ Other-service',
       'occupation_simplificada_ Priv-house-serv',
       'occupation_simplificada_ Prof-specialty',
       'occupation_simplificada_ Protective-serv',
       'occupation_simplificada_ Sales',
       'occupation_simplificada_ Tech-support',
       'occupation_simplificada_ Transport-moving',
       'marital-status_simplificada_ Never-married',
       'marital-status_simplificada_ No-spouse',
       'relationship_simplificada_ Other-relative',
       'relationship_simplificada_ Own-child',
       'relationship_simplificada_ Spouse',
       'relationship_simplificada_ Unmarried',
       'race_simplificada_ White',
       'native-country_simplificada_ America',
       'native-country_simplificada_ Asia',
       'native-country_simplificada_ Europe',
       'native-country_simplificada_ United-States']

print(len(variaveis))

In [ ]:
X = df_treino[variaveis]
y = df_treino['class_ >50K']